In [1]:
import numpy as np

import pandas as pd
from pandas import Series,DataFrame

In [2]:
df = DataFrame({'sex':np.random.randint(0,2,size = 300),'hand':np.random.randint(0,20,size =300),
                'IQ':np.random.randint(60,256,size = 300),'EQ':np.random.randint(30,256,size = 300)})
df

,sex,hand,IQ,EQ
0,1,9,87,223
1,1,3,74,42
2,0,9,63,242
3,1,1,186,233
4,1,8,100,139
...,...,...,...,...
295,0,1,202,243
296,1,18,192,229
297,1,17,246,155
298,1,15,138,50


In [3]:
df['sex'] = df['sex'].map({0:'男',1:'女'})

def convert(x):
    if x ==19:
        return 'left'
    return 'right'
df['hand'] = df['hand'].apply(convert)
df

,sex,hand,IQ,EQ
0,女,right,87,223
1,女,right,74,42
2,男,right,63,242
3,女,right,186,233
4,女,right,100,139
...,...,...,...,...
295,男,right,202,243
296,女,right,192,229
297,女,right,246,155
298,女,right,138,50


In [8]:
# 透视表和数据的分组聚合功能一样的
# 数据，往往很多，‘透视’-----------整理---------分组
pd.pivot_table(df,values=['IQ','EQ'],index = ['sex'],aggfunc=['mean','max','min'])

mean              max      min    
             EQ          IQ   EQ   IQ  EQ  IQ
sex                                          
女    134.911392  156.297468  255  253  30  60
男    143.112676  157.774648  255  255  32  60

In [11]:
df.groupby(by = ['sex'])['IQ','EQ'].agg([np.mean,'max','min']).stack(level = 0).unstack()

max             mean             min    
      EQ   IQ          EQ          IQ  EQ  IQ
sex                                          
女    255  253  134.911392  156.297468  30  60
男    255  255  143.112676  157.774648  32  60

In [12]:
# 透视表，指定了行和列
pd.pivot_table(df,values=['IQ','EQ'],index = ['sex'],columns=['hand'],aggfunc=['mean','max','min'])

mean              ...  min      
              EQ              ...   IQ      
hand        left       right  ... left right
sex                           ...           
女     150.857143  134.172185  ...   99    60
男     144.333333  143.000000  ...   65    60

[2 rows x 12 columns]

In [14]:
df.groupby(by = ['sex','hand'])['IQ','EQ'].agg([np.mean,'max','min']).unstack()

IQ                   ...    EQ           
            mean              max  ...   max  min      
hand        left       right left  ... right left right
sex                                ...                 
女     149.285714  156.622517  201  ...   255   64    30
男     138.833333  159.523077  243  ...   255   44    32

[2 rows x 12 columns]

In [18]:
# 算法预测
predict = np.load('./cancer_predict.npy',allow_pickle=True)
predict

array(['B', 'M', 'M', 'B', 'M', 'B', 'B', 'B', 'M', 'B', 'B', 'M', 'M',
       'B', 'B', 'M', 'B', 'B', 'B', 'B', 'B', 'B', 'M', 'M', 'B', 'B',
       'B', 'M', 'B', 'B', 'M', 'B', 'B', 'B', 'M', 'B', 'B', 'M', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'M', 'M', 'B', 'B', 'B', 'M',
       'M', 'B', 'M', 'M', 'B'], dtype=object)

In [19]:
# 真实的，确诊的
true = np.load('./cancer_true.npy',allow_pickle=True)
true

array(['B', 'M', 'M', 'B', 'M', 'B', 'B', 'B', 'M', 'B', 'B', 'M', 'M',
       'B', 'B', 'M', 'B', 'B', 'B', 'B', 'B', 'B', 'M', 'M', 'B', 'B',
       'B', 'M', 'M', 'M', 'M', 'B', 'B', 'B', 'M', 'B', 'B', 'M', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'M', 'M', 'B', 'B', 'B', 'M',
       'M', 'B', 'B', 'B', 'B'], dtype=object)

In [25]:
Series(predict).value_counts()

B    38
M    19
dtype: int64

In [22]:
# cross交叉，对比
# 真实数据
# 算法预测
pd.crosstab(index = predict,columns=true,rownames=['预测'],colnames=['确诊'],margins=True)

确诊,B,M,All
预测,,,
B,36,2,38
M,2,17,19
All,38,19,57
